In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 데이터 전처리 알고리즘
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

# 학습용과 검증용으로 나누는 함수
from sklearn.model_selection import train_test_split

# 교차 검증
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

# 평가함수
# 분류용
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

# 회귀용
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# 모델의 최적의 하이퍼 파라미터를 찾기 위한 도구
from sklearn.model_selection import GridSearchCV

# 머신러닝 알고리즘 - 분류
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier

# 머신러닝 알고리즘 - 회귀
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import VotingRegressor

# 학습 모델 저장을 위한 라이브러리
import pickle

# statsmodels API 불러오기
import statsmodels.api as sm

from sklearn.metrics import confusion_matrix, classification_report

In [2]:
# 데이터 로드 및 전처리
train_df = pd.read_csv("final_features.csv")
train_df = train_df.drop(columns=['ID'])
X = train_df.drop(columns=['Segment'])
y = train_df['Segment']

# 학습/검증 분할
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 레이블 인코딩 및 스케일링
encoder = LabelEncoder()
encoder.fit(y_train)
y_train_enc = encoder.transform(y_train)
y_val_enc   = encoder.transform(y_val)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)

In [3]:
# 모델 학습
model = XGBClassifier(tree_method='gpu_hist', predictor='gpu_predictor', random_state=42)
model.fit(X_train_scaled, y_train_enc)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              feature_weights=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [4]:
# 예측 및 평가
y_pred_enc = model.predict(X_val_scaled)
y_pred     = encoder.inverse_transform(y_pred_enc)

cm = confusion_matrix(y_val, y_pred, labels=encoder.classes_)
print("혼동행렬:")
print(pd.DataFrame(
    cm,
    index=[f"실제_{cls}" for cls in encoder.classes_],
    columns=[f"예측_{cls}" for cls in encoder.classes_]
))
print("\n분류 리포트:")
print(classification_report(y_val, y_pred, target_names=encoder.classes_))

혼동행렬:
      예측_A  예측_B  예측_C  예측_D   예측_E
실제_A     1     0    31     0      0
실제_B     0     0     5     0      0
실제_C     1     0  2137  1480    635
실제_D     1     0   802  6177   4662
실제_E     0     0   213  1944  61911

분류 리포트:
              precision    recall  f1-score   support

           A       0.33      0.03      0.06        32
           B       0.00      0.00      0.00         5
           C       0.67      0.50      0.57      4253
           D       0.64      0.53      0.58     11642
           E       0.92      0.97      0.94     64068

    accuracy                           0.88     80000
   macro avg       0.51      0.41      0.43     80000
weighted avg       0.87      0.88      0.87     80000

